In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365683 sha256=174e4bcfe406fffffe7edcd245a56bfbdfb4b1d4e92be630cdb352655c432ae0
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
from  mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [ ]:
def train(model, x_train, y_train, rank, size):
# Split the data across the nodes
    n = len(x_train)
    chunk_size = n // size
    start = rank * chunk_size 
    end = (rank + 1) * chunk_size
    if rank == size - 1:
        end = n
    x_train_chunk = x_train[start:end]
    y_train_chunk = y_train[start:end]
    model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
    #Train the model
    model.fit(x_train_chunk, y_train_chunk, epochs=1, batch_size=32)
    # Compute the accuracy on the training data
    train_loss, train_acc = model.evaluate(x_train_chunk, y_train_chunk, verbose=2)
    # Reduce the accuracy across all nodes
    train_acc = comm.allreduce(train_acc, op=MPI.SUM)
    return train_acc / size

In [ ]:
epochs = 5
for epoch in range(epochs):
# Train the model
    train_acc = train(model, x_train, y_train, rank, size)
    # Compute the accuracy on the test data
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    # Reduce the accuracy across all nodes
    test_acc = comm.allreduce(test_acc, op=MPI.SUM)
    # Print the results 
    if rank ==0:
        print(f"Epoch {epoch + 1}: Train accuracy = {train_acc:.4f}, Test accuracy = {test_acc /size:.4f}")

1875/1875 [==============================] - 39s 20ms/step - loss: 0.2040 - accuracy: 0.9419
1875/1875 - 11s - loss: 0.0772 - accuracy: 0.9785 - 11s/epoch - 6ms/step
313/313 - 2s - loss: 0.0779 - accuracy: 0.9762 - 2s/epoch - 6ms/step
Epoch 1: Train accuracy = 0.9785, Test accuracy = 0.9762
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0752 - accuracy: 0.9780
1875/1875 - 11s - loss: 0.0563 - accuracy: 0.9837 - 11s/epoch - 6ms/step
313/313 - 2s - loss: 0.0641 - accuracy: 0.9783 - 2s/epoch - 6ms/step
Epoch 2: Train accuracy = 0.9837, Test accuracy = 0.9783
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0574 - accuracy: 0.9829
1875/1875 - 11s - loss: 0.0442 - accuracy: 0.9873 - 11s/epoch - 6ms/step
313/313 - 2s - loss: 0.0578 - accuracy: 0.9807 - 2s/epoch - 6ms/step
Epoch 3: Train accuracy = 0.9873, Test accuracy = 0.9807
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0469 - accuracy: 0.9866
1875/1875 - 10s - loss: 0.0365 